Questo codice ha l'utilità di svolgere in automatico la media su 5 run di training differenti. Con questi esperimenti ripetuti si vuole fare un'analisi statistica delle prestazioni dei modelli.

## Git

In [ ]:
! git clone https://github.com/aripat/waste-sorting-CNN.git

Cloning into 'waste-sorting-CNN'...
remote: Enumerating objects: 15330, done.
remote: Counting objects: 100% (2767/2767), done.
remote: Compressing objects: 100% (2766/2766), done.
remote: Total 15330 (delta 1), reused 2766 (delta 1), pack-reused 12563
Receiving objects: 100% (15330/15330), 1.27 GiB | 18.01 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Checking out files: 100% (22641/22641), done.


## Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,  Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy, categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import os
import random

## Base dirs

In [ ]:
BASE_DIR = './waste-sorting-CNN/'
DATA_DIR = os.path.join(BASE_DIR, 'data')
os.listdir(BASE_DIR)

['README.md',
 'legenda.txt',
 'models',
 'data',
 'Trash test.pdf',
 'codice_utile',
 '.git']

# Set hyperparams

## Hyperparameters A

In [ ]:
# # #A
# WIDTH, HEIGHT = 256, 256
# CLASSES = ['cardboard', 'compost', 'glass', 'metal', 'paper', 'plastic', 'trash']
# PREPROCESSING = tf.keras.applications.vgg16.preprocess_input

# EPOCHS = 20
# BATCH_SIZE = 32
# LR = 1e-4

## Hyperparameters B

In [ ]:
# #B
WIDTH, HEIGHT = 299, 299
CLASSES = ['cardboard', 'compost', 'glass', 'metal', 'paper', 'plastic', 'trash']
PREPROCESSING = tf.keras.applications.inception_resnet_v2.preprocess_input

EPOCHS = 80
BATCH_SIZE = 32
LR = 1e-4

 # Dataset

## Set Generator/preprocessing func

In [ ]:
generator = ImageDataGenerator(
    preprocessing_function=PREPROCESSING
)

## Test sets

In [ ]:
path_sets = []
test_sets = []
#C
test0 = os.path.join(DATA_DIR, 'split_clean_7', 'test')
path_sets.append(test0)
#REAL
test1 = os.path.join(DATA_DIR, 'test_realistic_7')
path_sets.append(test1)
#BG
test2 = os.path.join(DATA_DIR, 'split_newBG_7', 'test')
path_sets.append(test2)
#C+BG
test3 = os.path.join(DATA_DIR, 'split_C+BG_7', 'test')
path_sets.append(test3)
#2XC
test4 = os.path.join(DATA_DIR, 'split_cleanX2_7', 'test')
path_sets.append(test4)

for path in path_sets:
  test_batches = generator.flow_from_directory(directory=path, target_size=(WIDTH, HEIGHT), classes=CLASSES, batch_size=BATCH_SIZE, shuffle=False)
  test_sets.append(test_batches)


Found 474 images belonging to 7 classes.
Found 113 images belonging to 7 classes.
Found 474 images belonging to 7 classes.
Found 948 images belonging to 7 classes.
Found 948 images belonging to 7 classes.


## Set training set 

In [ ]:
train_paths = ['clean', 'newBG', 'C+BG', 'cleanX2']
train_path = os.path.join(DATA_DIR, 'split_cleanX2_7', 'train')
X = generator.flow_from_directory(directory=train_path, target_size=(WIDTH, HEIGHT), classes=CLASSES, batch_size=BATCH_SIZE)

train_path

Found 3844 images belonging to 7 classes.


'./waste-sorting-CNN/data/split_cleanX2_7/train'

# Models

##Model A

In [ ]:

# X = train_batches, poiché è un "generator", i target y sono contenuti in X
def fit_model_A(X, test_sets):  
  # struttura e compile della rete
  model = Sequential([
    Conv2D(filters=96, kernel_size=(11, 11), activation='relu', padding='same', strides=(4,4), input_shape=(256, 256, 3)),
    MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='valid'),

    Conv2D(filters=192, kernel_size=(5, 5), activation='relu', padding='same', strides=(1,1)),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

    Conv2D(filters=288, kernel_size=(3, 3), activation='relu', padding='same', strides=(1,1)),
    Conv2D(filters=288, kernel_size=(3, 3), activation='relu', padding='same', strides=(1,1)),
    Conv2D(filters=192, kernel_size=(3, 3), activation='relu', padding='same', strides=(1,1)),
    MaxPool2D(pool_size=(3, 3), strides=2, padding='valid'),

    Flatten(),
    
    Dense(units=4096,  input_shape=(256, 256, 3), activation='relu'),
    Dense(units=4096, activation='relu'),
    Dense(units=7, activation='softmax') 
  ])
  model.compile(
      optimizer=Adam(learning_rate=LR), #
      loss='categorical_crossentropy', 
      metrics=['categorical_accuracy'])

  #fit della rete
  model.fit(X, steps_per_epoch=len(X), epochs=EPOCHS, verbose=0)
  
  #report_table: 2D dataframe con colonne=metriche, 1 riga= 1 test
  columns = ['accuracy', 'macro_f1', 'micro_f1']
  report_table = pd.DataFrame(columns=columns)

  #predict sui test set
  for X_test in test_sets:
    predictions = model.predict(X_test, verbose=0)
    report = classification_report(y_true=X_test.classes, y_pred=np.argmax(predictions, axis=-1), output_dict=True) ####
    accuracy = report['accuracy']
    macro_f1 = report['macro avg']['f1-score']
    micro_f1 =  report['weighted avg']['f1-score']
    # data = [[1row][2row]...]
    data = [[accuracy, macro_f1, micro_f1]]
    df = pd.DataFrame(data, columns=columns)
    report_table = report_table.append(df, ignore_index=True) # l'indexing continua da dove era rimasto

  return report_table, model




## Model B

In [ ]:
BASE_MODEL = tf.keras.applications.InceptionResNetV2(
    include_top=False, # we don't want the top layer (1000 image classifier)
    weights="imagenet", # transfer learning
)

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
def fit_model_B(X, test_sets):

  x = BASE_MODEL.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(7, activation='softmax')(x)

  model = Model(inputs=BASE_MODEL.input, outputs=predictions)

  for layer in BASE_MODEL.layers:
    layer.trainable = False

  model.compile(
      optimizer=Adam(learning_rate=LR), 
      loss='categorical_crossentropy', 
      metrics=['categorical_accuracy'])

  model.fit(X, steps_per_epoch=len(X), epochs=EPOCHS, verbose=0)
  
   #report_table: 2D dataframe con colonne=metriche, 1 riga= 1 test
  columns = ['accuracy', 'macro_f1', 'micro_f1']
  report_table = pd.DataFrame(columns=columns)

  #predict sui test set
  for X_test in test_sets:
    predictions = model.predict(X_test, verbose=0)
    report = classification_report(y_true=X_test.classes, y_pred=np.argmax(predictions, axis=-1), output_dict=True) ####
    accuracy = report['accuracy']
    macro_f1 = report['macro avg']['f1-score']
    micro_f1 =  report['weighted avg']['f1-score']
    # data = [[1row][2row]...]
    data = [[accuracy, macro_f1, micro_f1]]
    df = pd.DataFrame(data, columns=columns)
    report_table = report_table.append(df, ignore_index=True) # l'indexing continua da dove era rimasto

  return report_table, model


# Run repetitions

## Set numero ripetizioni, modello e training set

In [ ]:
N = 5
model = 'B'
training_sets = ['C', 'BG', 'C+BG', '2XC']
training_set = '2XC'

###Set fit

In [ ]:
# TODO::::::: prendere il report e fare anche le medie della f1 di ogni classe per A_C, B_C e B_C+BG

import datetime

tests = ['1::clean', '2::realistic', '3::bg_replm', '4::clean+bg', '5::cleanX2']
columns = ['test', 'accuracy', 'macro_f1', 'micro_f1']
report_table = pd.DataFrame(columns=columns[1:])

for i in range(N):
  report, modello = fit_model_B(X, test_sets) 
  report_table = report_table.append(report, ignore_index=False)  #non continua l'indexing ma ricomincia
  now = datetime.datetime.now()
  print('Run numero ' + str(i+1) + ' completata ----> ' + now.strftime("%Y-%m-%d %H:%M:%S"))
  print(report)
  print('*******************************************') 


Run numero 1 completata ----> 2021-06-09 10:57:58
   accuracy  macro_f1  micro_f1
0  0.852321  0.850034  0.851744
1  0.433628  0.444831  0.419843
2  0.578059  0.537405  0.571859
3  0.715190  0.697100  0.712329
4  0.854430  0.851182  0.853851
*******************************************
Run numero 2 completata ----> 2021-06-09 11:36:27
   accuracy  macro_f1  micro_f1
0  0.845992  0.842680  0.846046
1  0.451327  0.460181  0.444203
2  0.554852  0.522038  0.544048
3  0.700422  0.687589  0.697335
4  0.848101  0.842985  0.847932
*******************************************
Run numero 3 completata ----> 2021-06-09 12:15:14
   accuracy  macro_f1  micro_f1
0  0.864979  0.862048  0.863402
1  0.469027  0.486115  0.450629
2  0.537975  0.499342  0.526701
3  0.701477  0.686298  0.697758
4  0.864979  0.861159  0.863165
*******************************************
Run numero 4 completata ----> 2021-06-09 12:54:15
   accuracy  macro_f1  micro_f1
0  0.850211  0.844668  0.849066
1  0.460177  0.475912  0.452

In [ ]:
fname = model + '_' + training_set + '.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
report_table.to_csv(fname, index = True, header = True)


for i in range(N): 
   print('test ' + tests[i] + ' ************************')
   print(report_table[report_table.index==i])
   

test 1::clean ************************
   accuracy  macro_f1  micro_f1
0  0.852321  0.850034  0.851744
0  0.845992  0.842680  0.846046
0  0.864979  0.862048  0.863402
0  0.850211  0.844668  0.849066
0  0.843882  0.837120  0.843071
test 2::realistic ************************
   accuracy  macro_f1  micro_f1
1  0.433628  0.444831  0.419843
1  0.451327  0.460181  0.444203
1  0.469027  0.486115  0.450629
1  0.460177  0.475912  0.452382
1  0.442478  0.439591  0.426134
test 3::bg_replm ************************
   accuracy  macro_f1  micro_f1
2  0.578059  0.537405  0.571859
2  0.554852  0.522038  0.544048
2  0.537975  0.499342  0.526701
2  0.554852  0.524474  0.550493
2  0.561181  0.516735  0.546341
test 4::clean+bg ************************
   accuracy  macro_f1  micro_f1
3  0.715190  0.697100  0.712329
3  0.700422  0.687589  0.697335
3  0.701477  0.686298  0.697758
3  0.702532  0.688016  0.700632
3  0.702532  0.681812  0.696750
test 5::cleanX2 ************************
   accuracy  macro_f1  mi

In [ ]:
# report tables divise per test(index), con questi dati posso fare le statistiche sull'evaluation del modello
# TEST RESULTS

mean = pd.DataFrame(columns=columns[1:])
var = pd.DataFrame(columns=columns[1:])

# QUI
for i in range(N):
  results = report_table[report_table.index==i]
  
  df = results[columns[1:]].mean()
  mean = mean.append(df, ignore_index=True)

  df = results[columns[1:]].var(ddof=0)
  var = var.append(df, ignore_index=True)

# varUpper = mean + var
# varUpper.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varUp.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
# varUpper.to_csv(fname, index = False, header = True)

# varLower = mean - var
# varLower.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varLo.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
# varLower.to_csv(fname, index = False, header = True)

var.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_var.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
var.to_csv(fname, index = False, header = True)

mean.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_mean.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
mean.to_csv(fname, index = False, header = True)

print('var**************************\n')
print(var)
print()
# print('varUpper*********************\n')
# print(varUpper)
# print()
print('mean*************************\n')
print(mean)
print()
# print('varLower*********************\n')
# print(varLower)

var**************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.000054  0.000071  0.000049
1  2::realistic  0.000157  0.000315  0.000175
2   3::bg_replm  0.000166  0.000153  0.000209
3   4::clean+bg  0.000030  0.000025  0.000034
4    5::cleanX2  0.000035  0.000050  0.000030

mean*************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.851477  0.847310  0.850666
1  2::realistic  0.451327  0.461326  0.438639
2   3::bg_replm  0.557384  0.519999  0.547888
3   4::clean+bg  0.704430  0.688163  0.700961
4    5::cleanX2  0.855063  0.849956  0.854135



In [ ]:
modello.save('/content/drive/MyDrive/trash/models/' + model + '_' + training_set + '.h5')

# Medie e varianze

In [ ]:
import pandas as pd
import os

columns = ['test', 'accuracy', 'macro_f1', 'micro_f1']
tests = ['1::clean', '2::realistic', '3::bg_replm', '4::clean+bg', '5::cleanX2']

N = 5
model = 'A'
training_set = '2XC'
report_table = pd.read_csv("/content/drive/MyDrive/trash/batteria2/A_2XC.csv")

report_table.rename( columns={'Unnamed: 0':'test'}, inplace=True)
print(report_table)

    test  accuracy  macro_f1  micro_f1
0      0  0.662447  0.663681  0.662990
1      1  0.141593  0.129972  0.104107
2      2  0.265823  0.197723  0.238641
3      3  0.464135  0.460099  0.465370
4      4  0.674051  0.671778  0.676249
5      0  0.698312  0.688880  0.694892
6      1  0.141593  0.105935  0.102137
7      2  0.248945  0.169852  0.205489
8      3  0.473629  0.459729  0.468131
9      4  0.704641  0.695075  0.700794
10     0  0.753165  0.746323  0.752191
11     1  0.150442  0.130842  0.114864
12     2  0.246835  0.187015  0.218784
13     3  0.500000  0.494988  0.500727
14     4  0.738397  0.730172  0.737325
15     0  0.670886  0.663946  0.670166
16     1  0.176991  0.155825  0.141318
17     2  0.246835  0.174676  0.208927
18     3  0.458861  0.442596  0.453687
19     4  0.672996  0.666099  0.671042
20     0  0.702532  0.688479  0.694640
21     1  0.123894  0.102717  0.097400
22     2  0.236287  0.167833  0.201967
23     3  0.469409  0.454217  0.461437
24     4  0.697257  0.682

In [ ]:
mean = pd.DataFrame(columns=columns[1:])
var = pd.DataFrame(columns=columns[1:])


for i in range(N):
  results = report_table.loc[report_table['test'] == i]
  print(results)
  df = results[columns[1:]].mean()
  mean = mean.append(df, ignore_index=True)

  df = results[columns[1:]].var(ddof=0)
  var = var.append(df, ignore_index=True)


    test  accuracy  macro_f1  micro_f1
0      0  0.662447  0.663681  0.662990
5      0  0.698312  0.688880  0.694892
10     0  0.753165  0.746323  0.752191
15     0  0.670886  0.663946  0.670166
20     0  0.702532  0.688479  0.694640
    test  accuracy  macro_f1  micro_f1
1      1  0.141593  0.129972  0.104107
6      1  0.141593  0.105935  0.102137
11     1  0.150442  0.130842  0.114864
16     1  0.176991  0.155825  0.141318
21     1  0.123894  0.102717  0.097400
    test  accuracy  macro_f1  micro_f1
2      2  0.265823  0.197723  0.238641
7      2  0.248945  0.169852  0.205489
12     2  0.246835  0.187015  0.218784
17     2  0.246835  0.174676  0.208927
22     2  0.236287  0.167833  0.201967
    test  accuracy  macro_f1  micro_f1
3      3  0.464135  0.460099  0.465370
8      3  0.473629  0.459729  0.468131
13     3  0.500000  0.494988  0.500727
18     3  0.458861  0.442596  0.453687
23     3  0.469409  0.454217  0.461437
    test  accuracy  macro_f1  micro_f1
4      4  0.674051  0.671

In [ ]:
var.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_var.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
var.to_csv(fname, index = False, header = True)

mean.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_mean.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
mean.to_csv(fname, index = False, header = True)

print('var**************************\n')
print(var)
print()
print('mean*************************\n')
print(mean)
print()


# varUpper = mean + var
# varUpper.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varUp.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
# varUpper.to_csv(fname, index = False, header = True)

# print('varUpper*********************\n')
# print(varUpper)
# print()

# varLower = mean - var
# varLower.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varLo.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
# varLower.to_csv(fname, index = False, header = True)

# print('varLower*********************\n')
# print(varLower)

var**************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.001012  0.000909  0.000982
1  2::realistic  0.000301  0.000374  0.000248
2   3::bg_replm  0.000091  0.000128  0.000174
3   4::clean+bg  0.000204  0.000307  0.000262
4    5::cleanX2  0.000575  0.000518  0.000553

mean*************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.697468  0.690262  0.694976
1  2::realistic  0.146903  0.125059  0.111965
2   3::bg_replm  0.248945  0.179420  0.214762
3   4::clean+bg  0.473207  0.462326  0.469870
4    5::cleanX2  0.697468  0.689206  0.695258



## Deviazione standard

In [ ]:
columns = ['test', 'accuracy', 'macro_f1', 'micro_f1']
tests = ['1::clean', '2::realistic', '3::bg_replm', '4::clean+bg', '5::cleanX2']

N = 5
model = 'B'
training_set = 'C+BG'
var = pd.read_csv("/content/drive/MyDrive/trash/batteria2/B_C+BG_var.csv")

std = var[columns[1:]]**(1/2)
std.insert(0, columns[0], tests)
print(std)
fname = model + '_' + training_set + '_std.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
std.to_csv(fname, index = False, header = True)

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.004134  0.007323  0.004858
1  2::realistic  0.022666  0.028441  0.024948
2   3::bg_replm  0.005723  0.005255  0.006916
3   4::clean+bg  0.003913  0.004613  0.004341
4    5::cleanX2  0.004699  0.006938  0.005260


## Varianza e std corretta

In [ ]:
columns = ['test', 'accuracy', 'macro_f1', ' micro_f1']
tests = ['1::clean', '2::realistic', '3::bg_replm', '4::clean+bg', '5::cleanX2']

N = 5
model = 'B'
training_set = 'C'
var = pd.read_csv("/content/drive/MyDrive/trash/batteria2/var_std/B_C_var.csv")

print(var)
vcorr = var[columns[1:]]*(N)/(N-1)
vcorr.insert(0, columns[0], tests)
print(vcorr)
fname = model + '_' + training_set + '_vcorr.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2/corr', fname)
vcorr.to_csv(fname, index = False, header = True)

scorr = vcorr[columns[1:]]**(1/2)
scorr.insert(0, columns[0], tests)
print(scorr)
fname = model + '_' + training_set + '_scorr.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2/corr', fname)
scorr.to_csv(fname, index = False, header = True)

           test  accuracy  macro_f1   micro_f1
0      1::clean  0.000070  0.000100   0.000063
1  2::realistic  0.000107  0.000097   0.000139
2   3::bg_replm  0.000013  0.000041   0.000012
3   4::clean+bg  0.000030  0.000072   0.000030
4    5::cleanX2  0.000014  0.000021   0.000011
           test  accuracy  macro_f1   micro_f1
0      1::clean  0.000088  0.000125   0.000079
1  2::realistic  0.000133  0.000121   0.000174
2   3::bg_replm  0.000016  0.000051   0.000015
3   4::clean+bg  0.000038  0.000091   0.000037
4    5::cleanX2  0.000018  0.000026   0.000014
           test  accuracy  macro_f1   micro_f1
0      1::clean  0.009364  0.011197   0.008888
1  2::realistic  0.011538  0.011011   0.013184
2   3::bg_replm  0.004058  0.007155   0.003928
3   4::clean+bg  0.006151  0.009519   0.006093
4    5::cleanX2  0.004246  0.005078   0.003734


In [ ]:
import pandas as pd
import numpy as np

N=5
columns = ['test', 'accuracy', 'macro_f1', 'micro_f1']
report_table = pd.DataFrame(columns=columns[1:])

for i in range(N):
  df = pd.DataFrame(np.random.rand(5, 3), columns=columns[1:])
  report_table = report_table.append(df, ignore_index = False)

print(report_table)


   accuracy  macro_f1  micro_f1
0  0.265183  0.095616  0.587208
1  0.664928  0.685314  0.638157
2  0.256792  0.852933  0.323768
3  0.974306  0.131694  0.633723
4  0.918680  0.876583  0.171920
0  0.416665  0.429372  0.234067
1  0.469043  0.413419  0.655209
2  0.342575  0.602286  0.543493
3  0.896423  0.357850  0.443327
4  0.636941  0.578723  0.791606
0  0.605649  0.528298  0.107706
1  0.945077  0.989438  0.892581
2  0.086660  0.524695  0.201675
3  0.353591  0.418770  0.105509
4  0.043779  0.498433  0.542718
0  0.595633  0.088754  0.715079
1  0.948199  0.323121  0.634339
2  0.425356  0.701690  0.663721
3  0.554034  0.013562  0.300361
4  0.479184  0.339946  0.789563
0  0.309782  0.366816  0.041661
1  0.722694  0.547242  0.114043
2  0.344892  0.241162  0.953006
3  0.275125  0.645963  0.660167
4  0.191484  0.572668  0.536172


In [ ]:
columns = ['test', 'accuracy', 'macro_f1', 'micro_f1']
tests = ['1::clean', '2::realistic', '3::bg_replm', '4::clean+bg', '5::cleanX2']
mean = pd.DataFrame(columns=columns[1:])
var = pd.DataFrame(columns=columns[1:])

# QUI
for i in range(N):
  results = report_table[report_table.index==i]
  
  df = results[columns[1:]].mean()
  mean = mean.append(df, ignore_index=True)

  df = results[columns[1:]].var(ddof=0)
  var = var.append(df, ignore_index=True)

varUpper = mean + var
varUpper.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_varUp.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
varUpper.to_csv(fname, index = False, header = True)

varLower = mean - var
varLower.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_varLo.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
varLower.to_csv(fname, index = False, header = True)

var.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_var.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
var.to_csv(fname, index = False, header = True)

mean.insert(0, columns[0], tests)
fname = model + '_' + training_set + '_mean.csv'
fname = os.path.join('/content/drive/MyDrive/trash/batteria2', fname)
mean.to_csv(fname, index = False, header = True)

print('mean*************************\n')
print(mean)
print()
print('var**************************\n')
print(var)
print()
print('varUpper*********************\n')
print(varUpper)
print()
print('varLower*********************\n')
print(varLower)

mean*************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.438582  0.301771  0.337144
1  2::realistic  0.749988  0.591707  0.586866
2   3::bg_replm  0.291255  0.584553  0.537133
3   4::clean+bg  0.610696  0.313568  0.428618
4    5::cleanX2  0.454014  0.573271  0.566396

var**************************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.019943  0.031941  0.071189
1  2::realistic  0.032852  0.054571  0.065316
2   3::bg_replm  0.013308  0.041513  0.069414
3   4::clean+bg  0.079155  0.049319  0.043350
4    5::cleanX2  0.097445  0.030414  0.051522

varUpper*********************

           test  accuracy  macro_f1  micro_f1
0      1::clean  0.458525  0.333712  0.408333
1  2::realistic  0.782840  0.646278  0.652182
2   3::bg_replm  0.304563  0.626066  0.606547
3   4::clean+bg  0.689851  0.362887  0.471967
4    5::cleanX2  0.551459  0.603685  0.617918

varLower*********************

           test  accuracy  macro_f1  micro_f1

In [ ]:
varLower.to_csv('/content/drive/MyDrive/trash/batteria2/prova2.csv', index = False, header = True)

lknknlnkjb

In [ ]:
import pandas as pd

columns = ['accuracy', 'macro_f1', 'micro_f1']
report_table = pd.DataFrame(columns=columns)

report = pd.read_csv("/content/drive/MyDrive/trash/report/report_A_2XC_T1.csv").transpose()
accuracy = report[7]['f1-score']
macro_f1 = report[8]['f1-score']
micro_f1 =  report[9]['f1-score']

# data = [[1row][2row]...]
data = [[accuracy, macro_f1, micro_f1]]
df = pd.DataFrame(data, columns=columns)
report_table = report_table.append(df, ignore_index=True)

report = pd.read_csv("/content/drive/MyDrive/trash/report/report_A_2XC_T2.csv").transpose()
accuracy = report[7]['f1-score']
macro_f1 = report[8]['f1-score']
micro_f1 =  report[9]['f1-score']

data = [[accuracy, macro_f1, micro_f1]]
df = pd.DataFrame(data, columns=columns)
report_table = report_table.append(df, ignore_index=True)
report_table

################################

columns = ['accuracy', 'macro_f1', 'micro_f1']
report_table1 = pd.DataFrame(columns=columns)

report = pd.read_csv("/content/drive/MyDrive/trash/report/report_A_2XC_T1.csv").transpose()
accuracy = report[7]['f1-score']
macro_f1 = report[8]['f1-score']
micro_f1 =  report[9]['f1-score']

# data = [[1row][2row]...]
data = [[accuracy, macro_f1, micro_f1]]
df = pd.DataFrame(data, columns=columns)
report_table1 = report_table1.append(df, ignore_index=True)

report = pd.read_csv("/content/drive/MyDrive/trash/report/report_A_2XC_T2.csv").transpose()
accuracy = report[7]['f1-score']
macro_f1 = report[8]['f1-score']
micro_f1 =  report[9]['f1-score']

data = [[accuracy, macro_f1, micro_f1]]
df = pd.DataFrame(data, columns=columns)
report_table1 = report_table1.append(df, ignore_index=True)

# print(report_table)
# print()
# print(report_table1)

report_table1 = report_table1.append(report_table, ignore_index=False)
print(report_table1)

test0 = report_table1[report_table1.index==0]
print(test0)

   accuracy  macro_f1   micro_f1
0  0.723629  0.710364   0.719028
1  0.185841  0.158844   0.136730
0  0.723629  0.710364   0.719028
1  0.185841  0.158844   0.136730
   accuracy  macro_f1   micro_f1
0  0.723629  0.710364   0.719028
0  0.723629  0.710364   0.719028


In [ ]:
import pandas as pd

# TODO:   inserire il codice QUI  

tests = ['clean', 'realistic', 'bg_replm', 'clean+bg', 'cleanX2']
mean = pd.read_csv('/content/drive/MyDrive/trash/batteria/B_C+BG.csv')
mean['test'] = tests
columns = ['test','accuracy', 'macro_f1', 'micro_f1']
var = mean[columns[1:]].var(axis=0)
# fname = model + '_' + training_set + '_var.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/var', fname)
# var.to_csv(fname, index = False, header = True)

varUpper = mean[columns[1:]] + var
varUpper.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varUp.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/var', fname)
# varUpper.to_csv(fname, index = False, header = True)

varLower = mean[columns[1:]] - var
varLower.insert(0, columns[0], tests)
# fname = model + '_' + training_set + '_varLo.csv'
# fname = os.path.join('/content/drive/MyDrive/trash/var', fname)
# varLower.to_csv(fname, index = False, header = True)

print(var)
print(mean)
print(varUpper)
print(varLower)


accuracy     0.022809
macro_f1     0.022080
 micro_f1    0.023440
dtype: float64
        test  accuracy  macro_f1   micro_f1
0      clean  0.837553  0.821824   0.835425
1  realistic  0.483186  0.474518   0.476330
2   bg_replm  0.637975  0.619750   0.632166
3   clean+bg  0.737764  0.722356   0.734559
4    cleanX2  0.842616  0.826278   0.840241
        test  accuracy  macro_f1   micro_f1
0      clean  0.860362  0.843904   0.858865
1  realistic  0.505995  0.496598   0.499771
2   bg_replm  0.660784  0.641830   0.655606
3   clean+bg  0.760573  0.744436   0.757999
4    cleanX2  0.865425  0.848358   0.863681
        test  accuracy  macro_f1   micro_f1
0      clean  0.814744  0.799744   0.811985
1  realistic  0.460377  0.452438   0.452890
2   bg_replm  0.615166  0.597670   0.608726
3   clean+bg  0.714955  0.700276   0.711118
4    cleanX2  0.819807  0.804197   0.816800
